In [6]:

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # To convert them into chunks
from langchain.llms import CTransformers  # For quantized models

In [7]:
PINECONE_API_KEY="fc01720d-4061-4e31-90ac-f6efc4dd8056"
PINECONE_API_ENV="gcp-starter"

In [ ]:
# Extracting the data

def laod_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

